In [1]:
# Import necessary packages:

import cobra
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy
from datetime import datetime, timedelta
from Light import TotSolEng
from scipy.interpolate import RegularGridInterpolator
import netCDF4
from IPython.display import clear_output

In [2]:
# Initialize a dictionary to hold all inputs to the class.
Inputs = {}

# User defined inputs here!!! Names will be more descriptive in this user-facing section.
Inputs['Year'] = 2023
Inputs['DayNumber'] = 0
Inputs['Month'] = 3 # Month 0 is a flag for not using MM/DD/YYYY mode.
Inputs['Day'] = 1 # Day 0 is a flag for not using MM/DD/YYYY mode.
Inputs['Hour'] = 12 # 24-hour time
Inputs['Minute'] = 0
Inputs['Second'] = 0
Inputs['Interval'] = 0 # Defaults to zero. Do not change, interval will be handed in dFBA.
Inputs['Latitude'] = 75 # Bottom of Arctic Circle
Inputs['Longitude'] = 0 # Prime Meridian
Inputs['Timezone'] = 0 # We are at the prime meridian. (This will likely be automatically calculated.)
Inputs['Pressure'] = 1013 # DEFAULT: 1013 mb = 1 atmosphere
Inputs['Temperature'] = 0 # DEFAULT: 10 deg C (arctic will be much colder)

Inputs['Turbidity'] = 0.084
Inputs['Water Vapor'] = 1.4164
Inputs['Ozone'] = 0.3438
Inputs['Albedo'] = 0.1

# Input wavelength-dependent data tables.
Inputs['WVL-ETR'] = np.loadtxt('./Data/WVL-ETR.csv',delimiter=',',dtype='float64', encoding='utf-8-sig')
Inputs['WVL-ABS'] = np.loadtxt('./Data/WVL-ABS.csv',delimiter=',',dtype='float64', encoding='utf-8-sig')

Inputs["NO3flag"] = 0
Inputs["SiOH4flag"] = 0

start = str(Inputs['Month']).zfill(2) + "/" + str(Inputs['Day']).zfill(2) + "/" + str(Inputs['Year']) + " " + str(Inputs['Hour'])
start_date = datetime.strptime(start, "%m/%d/%Y %H")

In [3]:
def par(DTOT,allWVL):
    absorbML = {'EX_photon410_e': 4730.075289,'EX_photon430_e': 5817.128965,'EX_photon450_e': 5348.203973,'EX_photon470_e': 4050.000013,'EX_photon490_e': 3464.694801,'EX_photon510_e': 2649.794528,'EX_photon530_e': 1876.490736,'EX_photon550_e': 1334.544022,'EX_photon570_e': 873.4095179,'EX_photon590_e': 740.7816246,'EX_photon610_e': 888.7175101,'EX_photon630_e': 1082.718272,'EX_photon650_e': 1178.924274,'EX_photon670_e': 3322.974688,'EX_photon690_e': 1840.91646}
    Ipts = np.linspace(410,690,15)
    chla = 0.192e-9 # g chlA
    DW = 16.62883198e-12 # g DW
    conv = (60*60)/1e7 #to mmol gDW-1 h-1
    
    I = []
    for idx, group in enumerate(Ipts):
        bottom = Ipts[idx] - 10
        top = Ipts[idx] + 10
        idbottom = np.flatnonzero(allWVL == bottom)[0]
        idtop = np.flatnonzero(allWVL == top)[0]
        I.append(scipy.integrate.simpson(DTOT[idbottom:idtop],allWVL[idbottom:idtop]))
        
    pfa = {f:I[idx]*absorbML[f]*conv*(chla/DW) for idx,f in enumerate(absorbML.keys())}
    return pfa

In [4]:
def flattenlat(latmap):

    inflat = []
    
    for x in latmap:
        for y in list(x[0]):
            inflat.append(y)
    
    return inflat   

def listarraytoarray(inarray):

    inflat = []
    
    for x in inarray:
        for y in list(x):
            inflat.append(y)
    
    return inflat  

def addhrs(startdate,hrs):
    date = startdate + timedelta(hours=hrs)
    return date

def henry(T):
    
    T += 273.15 

    H0 = [3.3e-4, # CO2
          1.3e-5, # O2
          6.4e-6]
    
    dlnH = [2400, # CO2
            1500, # O2
            1400] # N2
    
    Henry = [] 
    
    for i in range(3):
        Henryin = H0[i]*math.exp(dlnH[i]*((1/T)-(1/298.15)))*101325000 # Converting to umol/(L*atm).
        Henry.append(Henryin)
    
    return Henry

def RLPD(CO2,O2,T):
    
    TK = T + 273.15
    R = 8.314
    vmax25 = 10.45 # umol CO2/mgDW*h
    Ko25 = 1500
    KoC3_25 = 311.6
    Koscale = 1500/311.6
    ## Temperature Dependent Params for:
        ##    Kc     Ko     kcat
    scale = [ 21.1,  19.7, 30.8 ] 
    dHa =   [43000,  34600, 76300 ]
    
    ## Q10 temperature-dependence for vmax:
    
    Kc = math.exp(scale[0] - (dHa[0]/(R*TK))) # parameters are in uM
    Ko = math.exp(scale[1] - (dHa[1]/(R*TK)))*Koscale # parameters are in uM
    kcat = math.exp(scale[2] - (dHa[2]/(R*TK)))
    vmax = kcat*vmax25 # umol/mgDW*h
    rate = (vmax*CO2)/(Kc*(1+(O2/Ko))+CO2) # umol/mgDW*h
    
    return rate

def RLPC(CO2,O2,T):
    
    TK = T + 273.15
    R = 8.314
    vmax25 = 3.68 # umol CO2/mgDW*h
    Ko25 = 199
    KoC3_25 = 311.6
    Koscale = 199/311.6
    ## Temperature Dependent Params for:
        ##    Kc     Ko     kcat
    scale = [ 20.8,  19.7, 14.2 ] 
    dHa =   [38800,  34600, 35200 ]
    
    ## Q10 temperature-dependence for vmax:
    
    Kc = math.exp(scale[0] - (dHa[0]/(R*TK))) # parameters are in uM
    Ko = math.exp(scale[1] - (dHa[1]/(R*TK)))*Koscale # parameters are in uM
    kcat = math.exp(scale[2] - (dHa[2]/(R*TK)))
    vmax = kcat*vmax25 # umol/mgDW*h
    rate = (vmax*CO2)/(Kc*(1+(O2/Ko))+CO2) # umol/mgDW*h
    
    return rate

In [5]:
# Create an object that will store data for wavelength-dependent light transmission through the atmosphere.
ICalc = TotSolEng(Inputs)
# Calculate the total incident extraterrestrial radiation (ETR) based on date, time and location.
ICalc.CalcTotSolarEng(Inputs)
# Calculate wavelength-dependent light transmission based on ETR spectrum.
ICalc.CalcSolarSpec(Inputs)

# Calculate wavelength-dependent light absorption/scattering by the ocean at a certain depth.
ICalc.waterabs(0)
ICalc.DTOTphoto = np.asarray(ICalc.DTOTphoto).squeeze()

# Integrate only the photoactive region of the incident spectrum to get total photoactive light intensity.
Iphoto = par(ICalc.DTOTphoto,ICalc.allWVLphoto)

Itot = scipy.integrate.simpson(ICalc.DTOTphoto,ICalc.allWVLphoto)

In [6]:
def LoadModel(): # model: cyanobacterium Synechocystis sp. PCC 6803; model2: diatom Phaeodactylum tricornutum CCAP 1055/1
    model = cobra.io.read_sbml_model('Models/Thaps_ML_model_n.xml')
    model2 = cobra.io.read_sbml_model('Models/iAnC892.xml')

    refmodel = model2
    ML = model.merge(model2,objective='sum')
    ML.objective = {ML.reactions.get_by_id("DM_biomass_c"):1,ML.reactions.get_by_id("biomass_eq_33047__vc"):1}

    # Create reaction objects for each of the new reactions we are importing.
    # The first reaction is created implicitly below as an exchange flux for stored chryso.
    # The two here are a "transport" reaction, unstoring the chryso, and a conversion directly to glucose.
    nh4T = cobra.Reaction('nh4T')
    carbonT = cobra.Reaction('carbonT')
    G6PD_c = cobra.Reaction('G6PD_c')
    
    ML.add_reactions([nh4T,carbonT])
    ML.reactions.carbonT.build_reaction_from_string('glc__D_c --> cpd00027[vc]')
    ML.reactions.nh4T.build_reaction_from_string('cpd00013[hc] --> nh4_c')
    
    ML.add_reactions([G6PD_c])
    ML.reactions.G6PD_c.build_reaction_from_string('g6p_c --> glc__D_c')
    
    ML.reactions.nh4T.lower_bound = -1000
    ML.reactions.nh4T.upper_bound = 1000
    ML.reactions.carbonT.lower_bound = -1000
    ML.reactions.carbonT.upper_bound = 1000

    # Create reaction objects for each of the new reactions we are importing.
    # The first reaction is created implicitly below as an exchange flux for stored chryso.
    # The two here are a "transport" reaction, unstoring the chryso, and a conversion directly to glucose.
    gluconv = cobra.Reaction('gluconv')
    chrysoT = cobra.Reaction('chrysoT')
    GLCt1 = cobra.Reaction('GLCt1')
    
    # Create metabolites to represent stored (represented as "extracellular") and available chryso.
    model.add_metabolites([
        cobra.Metabolite(
        'chryso_c',
        name='chrysolaminarin',
        compartment='c'
        ),
        cobra.Metabolite(
        'chryso_e',
        name='chrysolaminarin',
        compartment='e'
        ),
    ])
    
    ML.add_metabolites([
        cobra.Metabolite(
        'chryso_c',
        name='chrysolaminarin',
        compartment='c'
        ),
        cobra.Metabolite(
        'chryso_e',
        name='chrysolaminarin',
        compartment='e'
        ),
        cobra.Metabolite(
        'glc__D_e',
        name='D-Glucose',
        compartment='e'
        ),
    ])

    ML.add_reactions([GLCt1,chrysoT,gluconv])
    ML.add_boundary(ML.metabolites.get_by_id("glc__D_e"), type="exchange")
    ML.add_boundary(model.metabolites.get_by_id("chryso_e"), type="exchange")
    ML.reactions.gluconv.build_reaction_from_string('chryso_c --> glc__D_c')
    ML.reactions.chrysoT.build_reaction_from_string('chryso_e --> chryso_c')
    ML.reactions.GLCt1.build_reaction_from_string('glc__D_e --> glc__D_c')

    # Remove author's biomass constraint forcing a growth rate at or above observed.
    # If this biomass constraint is used, all solutions at low light are infeasible.
    ML.reactions.biomass_eq_33047__hc.lower_bound = 0

    # Must run this cell or it will rely entirely on heterotrophic growth!!!
    # Defaults for co2 exchange are ub = 0, lb = 0.
    ML.reactions.EX_co2_e.upper_bound = 1000
    ML.reactions.EX_co2_e.lower_bound = -1000
    
    # Must run this cell in order for photosynthesis to occur!
    # Defaults for light intensity are ub = 0, lb = 0. 
    # ML.reactions.EX_photon_e.lower_bound = -0.62
    # ML.reactions.EX_photon_e.upper_bound = 1000
    
    # Run this cell if you wish to perform autotrophic growth only.
    ML.reactions.EX_glc__D_e.lower_bound = 0
    ML.reactions.EX_glc__D_e.upper_bound = 0
    
    # Shut down or turn on the chrysolaminarin reaction.
    ML.reactions.EX_chryso_e.lower_bound = 0
    
    # Weird photon input of this model.
    for k in Iphoto.keys():
        ML.reactions.get_by_id(k).upper_bound = 0
        ML.reactions.get_by_id(k).lower_bound = Iphoto[k]*-1.
        ML.reactions.get_by_id(k).upper_bound = Iphoto[k]*-0.9999
    
    ML.reactions.EX_no3_e.lower_bound = -0.001
    ML.reactions.EX_no2_e.lower_bound = 0
    ML.reactions.EX_nh4_e.lower_bound = 0
    ML.reactions.EX_pi_e.lower_bound = -1
    ML.reactions.EX_so4_e.lower_bound = -1
    ML.reactions.EX_sio4h4_e.lower_bound = -1
    ML.reactions.EX_cncbl3_e.lower_bound = -1
    ML.reactions.EX_co2_e.lower_bound = -10
    ML.reactions.EX_hco3_e.lower_bound = -0.1

    datamin_re = np.loadtxt("CarbonTransfer.csv",delimiter=',')
    Tn = np.linspace(-3,26,30)
    NO3 = np.linspace(0,.5,3)
    Sili = np.linspace(0.2,2,18)
    datamin = datamin_re.reshape(datamin_re.shape[0], datamin_re.shape[1] // len(Sili), len(Sili))
    IYSi = RegularGridInterpolator((Tn,NO3),datamin[:,:,-1])
    INSi = RegularGridInterpolator((Tn,NO3,Sili),datamin)
    
    return ML, refmodel, IYSi, INSi

In [7]:
# Calculate the saturation concentration of carbon dioxide (in mM).
ML, refmodel, IYSi, INSi = LoadModel()

Henry = henry(12)
CO2sat = Henry[0]*(4.21e-4) # Partial pressure of CO2 in atm. # convert to uM
O2sat = Henry[1]*(.2105) # convert to uM
N2sat = Henry[2]*(.781)

y = [0, 0, CO2sat*4, 10, 10]
### DIATOM LIGHT ABSORPTION CALCULATIONS.
for k in Iphoto.keys():
    ML.reactions.get_by_id(k).upper_bound = 0
    ML.reactions.get_by_id(k).lower_bound = Iphoto[k]*-1.
    ML.reactions.get_by_id(k).upper_bound = Iphoto[k]*-0.9999

### DIATOM LIMITING RESOURCES CALCULATIONS. 
# Defaults for co2 exchange are ub = 0, lb = 0. Unbounded for now.
ML.reactions.EX_co2_e.upper_bound = 1000
ML.reactions.EX_co2_e.lower_bound = -RLPD(CO2sat*4,O2sat,5)
hco3 = 53.70*CO2sat # buffer calculation for hco3, assuming pH = 8.1 with pKa = 6.37 [10^(8.1-6.37) = 53.70]
chla = 0.192e-9
DW = 16.62883198e-12
n = 1.9
#ML.reactions.EX_hco3_e.lower_bound = -((2.438*hco3)/(258.6+hco3)) # Trimborn, 2009
ML.reactions.EX_hco3_e.lower_bound = 0
ML.reactions.EX_no3_e.lower_bound = -((1.474*y[3])/(6.14+y[3]))

if np.isnan(-((1.961*(y[4]**n))/((8.1**n)+(y[4]**n)))):
    pass
else:    
    ML.reactions.EX_sio4h4_e.lower_bound = -((1.961*(y[4]**n))/((8.1**n)+(y[4]**n)))

### CYANOBACTERIA CALCULATIONS

# Remove author's biomass constraint forcing a growth rate at or above observed.
# If this biomass constraint is used, all solutions at low light are infeasible.
ML.reactions.biomass_eq_33047__hc.lower_bound = 0

# Rubisco-limited photosynthesis 
ML.reactions.get_by_id("EX_cpd00242[e]").lower_bound = 0

# Rubisco-limited photosynthesis 
ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound = -RLPC(y[2],O2sat,5)
#ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound = 0

# Nitrogen fixation, including boost by being symbiotic with diatom...
ML.reactions.get_by_id("EX_cpd00528[e]").lower_bound = -(0.9*N2sat)/(165+N2sat)

ML.reactions.get_by_id("EX_chryso_e").lower_bound = 0

refmodel.reactions.get_by_id("EX_cpd00242[e]").lower_bound = 0
refmodel.reactions.get_by_id("EX_cpd00011[e]").lower_bound = -RLPC(y[2],O2sat,5)
refmodel.reactions.get_by_id("EX_cpd00528[e]").lower_bound = -(0.9*N2sat)/(165+N2sat)
refmodel.reactions.biomass_eq_33047__hc.lower_bound = 0

#ML.reactions.get_by_id("EX_cpd00209[e]").lower_bound = -1000 

In [8]:
refsolution = refmodel.optimize()

print(refsolution)
refmodel.summary()

<Solution 0.006 at 0x2190fab7a60>


Metabolite,Reaction,Flux,C-Number,C-Flux
hvphoton1[e],EX_PHO1,2.415,0,0.00%
hvphoton2[e],EX_PHO2,1.302,0,0.00%
cpd00001[e],EX_cpd00001[e],0.203,0,0.00%
cpd00009[e],EX_cpd00009[e],0.002455,0,0.00%
cpd00011[e],EX_cpd00011[e],0.2667,1,100.00%
cpd00034[e],EX_cpd00034[e],1.865E-05,0,0.00%
cpd00048[e],EX_cpd00048[e],0.002212,0,0.00%
cpd00058[e],EX_cpd00058[e],1.865E-05,0,0.00%
cpd00063[e],EX_cpd00063[e],2.794E-05,0,0.00%
cpd00067[e],EX_cpd00067[e],0.3455,0,0.00%


In [ ]:
Itot = 0.1

In [11]:
Imax = ML.problem.Constraint(
    (ML.reactions.EX_PHO1.flux_expression + ML.reactions.EX_PHO2.flux_expression),
    lb=-Itot,
    ub=0)
ML.add_cons_vars(Imax)

In [ ]:
ML.remove_cons_vars(Imax)

In [12]:
BioConstraint = ML.problem.Constraint(
    (ML.reactions.DM_biomass_c.flux_expression - ML.reactions.biomass_eq_33047__vc.flux_expression),
    lb=0,
    ub=0)
ML.add_cons_vars(BioConstraint)

In [ ]:
ML.remove_cons_vars(BioConstraint)

In [13]:
NonTrivial = ML.problem.Constraint(
    (ML.reactions.DM_biomass_c.flux_expression - ML.reactions.biomass_eq_33047__vc.flux_expression),
    lb=1e-6,
    ub=1000)
ML.add_cons_vars(NonTrivial)

In [ ]:
ML.remove_cons_vars(NonTrivial)

In [14]:
Tn = np.linspace(-3,26,30)
datamin_re = np.loadtxt("CarbonTransfer.csv",delimiter=',')
Tn = np.linspace(-3,26,30)
NO3 = np.linspace(0,.5,3)
Sili = np.linspace(0.2,2,18)
datamin = datamin_re.reshape(datamin_re.shape[0], datamin_re.shape[1] // len(Sili), len(Sili))
IYSi = RegularGridInterpolator((Tn,NO3),datamin[:,:,-1])
INSi = RegularGridInterpolator((Tn,NO3,Sili),datamin)
CT = np.interp(5,Tn,datamin[:,-1,-1])
print(CT)
CarbonConstraint = ML.problem.Constraint(
    ML.reactions.carbonT.flux_expression,
    lb=-(CT/6)*ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound, # 83% of metabolic C comes from diatom
    ub=-(CT/6)*ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound)
ML.add_cons_vars(CarbonConstraint)

1.2374616615465108


In [ ]:
ML.remove_cons_vars(CarbonConstraint)

In [17]:
AmmoniaConstraint = ML.problem.Constraint(
    ML.reactions.nh4T.flux_expression,
    lb=0*refsolution.fluxes["EX_cpd00528[e]"],
    ub=-4.5*refsolution.fluxes["EX_cpd00528[e]"])
ML.add_cons_vars(AmmoniaConstraint)

In [ ]:
ML.remove_cons_vars(AmmoniaConstraint)

In [9]:
# Rubisco-limited photosynthesis 
ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound = 0
#ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound = 0

# Nitrogen fixation, including boost by being symbiotic with diatom...
ML.reactions.get_by_id("EX_cpd00528[e]").lower_bound = -(0.9*N2sat)/(165+N2sat)

ML.reactions.get_by_id("EX_chryso_e").lower_bound = -0.016

for k in Iphoto.keys():
    ML.reactions.get_by_id(k).upper_bound = 0
    ML.reactions.get_by_id(k).lower_bound = -0.01
    ML.reactions.get_by_id(k).upper_bound = 0

ML.reactions.EX_hco3_e.lower_bound = 0
ML.reactions.EX_co2_e.lower_bound = 0
ML.reactions.EX_no3_e.lower_bound = -1000

ML.reactions.nh4T.lower_bound = 0
ML.reactions.nh4T.upper_bound = 1000
ML.reactions.carbonT.lower_bound = 0
ML.reactions.carbonT.upper_bound = 1000

ML.reactions.get_by_id("EX_cpd00209[e]").lower_bound = -((1.474*y[4])/(6.14+y[4]))

In [21]:
solution = ML.optimize()

print(solution)
ML.summary()

<Solution 0.031 at 0x158b087bdf0>


C:\Users\josep\miniconda3\envs\dFBA\lib\site-packages\cobra\core\metabolite.py:146: UserWarning: 0.0110442039994 is not an integer (in formula Mg0.0110442039994H63.6154800953Si0.49210312671C39.6884189474O23.5582872123N3.4699496483P0.353545674641S0.238990589691)
  warn(f"{count} is not an integer (in formula {self.formula})")
C:\Users\josep\miniconda3\envs\dFBA\lib\site-packages\cobra\core\metabolite.py:146: UserWarning: 63.6154800953 is not an integer (in formula Mg0.0110442039994H63.6154800953Si0.49210312671C39.6884189474O23.5582872123N3.4699496483P0.353545674641S0.238990589691)
  warn(f"{count} is not an integer (in formula {self.formula})")
C:\Users\josep\miniconda3\envs\dFBA\lib\site-packages\cobra\core\metabolite.py:146: UserWarning: 0.49210312671 is not an integer (in formula Mg0.0110442039994H63.6154800953Si0.49210312671C39.6884189474O23.5582872123N3.4699496483P0.353545674641S0.238990589691)
  warn(f"{count} is not an integer (in formula {self.formula})")
C:\Users\josep\minicond

Metabolite,Reaction,Flux,C-Number,C-Flux
hvphoton1[e],EX_PHO1,3.742,0,0.00%
hvphoton2[e],EX_PHO2,1.893,0,0.00%
co2_e,EX_co2_e,0.9624,1,76.92%
cpd00001[e],EX_cpd00001[e],0.1345,0,0.00%
cpd00009[e],EX_cpd00009[e],0.00595,0,0.00%
cpd00011[e],EX_cpd00011[e],0.2888,1,23.08%
cpd00034[e],EX_cpd00034[e],4.52E-05,0,0.00%
cpd00048[e],EX_cpd00048[e],0.005362,0,0.00%
cpd00058[e],EX_cpd00058[e],4.52E-05,0,0.00%
cpd00063[e],EX_cpd00063[e],6.773E-05,0,0.00%


In [ ]:
root = scipy.optimize.fsolve(NO3func,1,args=(-solution.fluxes["EX_no3_e"]))[0]
root

In [ ]:
test4 = solution.fluxes["DM_biomass_c"]

In [ ]:
test/test4

In [10]:
def NO3func(x,NO3max):
    return -((1.474*x)/(6.14+x)) + NO3max

In [11]:
def SiOH4func(x,SiOH4max):
    return -((1.961*(x**n))/((8.1**n)+(x**n))) + SiOH4max

In [12]:
# Differential Equation function for IVP solver:
"""
y:
0  [     D_Biomass    ]
1  [     C_Biomass    ]
2  [     D_Chrysol    ]
3  [  Carbon Dioxide  ]
4  [     Nitrate      ]
5  [     Silicon      ]
6  [   Total Biomass  ]
7  [    Total CO2     ]
8  [    Total O2      ]
9  [    Total N2      ]
"""

def dydt(t,y,Inputs,start_date,temp,IYSi,INSi,Tn,datamin):
    
    global daycounter
    global StartBool
    global FirstDay
    global FirstNight
    global FirstLight

    ML.solver.configuration.verbosity = 3
    ML.solver.configuration.timeout = 5
    ML.solver.configuration.tolerance_feasibility = 1e-6
    elapsedday = math.floor(t/24)
    T = temp[elapsedday]
    # Resolve time; input is minutes from start, need (year, day number, HH:MM:SS)
    date = addhrs(start_date,t)
    # Convert date object with added time to individual values and update all light calculator values.
    Inputs['Year'] = date.year
    Inputs['DayNumber'] = date.timetuple().tm_yday
    Inputs['Hour'] = date.hour
    Inputs['Minute'] = date.minute
    Inputs['Second'] = date.second

    # Calculate the saturation concentration of carbon dioxide (in mM).
    Henry = henry(T)
    CO2sat = Henry[0]*(4.21e-4) # Partial pressure of CO2 in atm. # convert to uM
    O2sat = Henry[1]*(.2105) # convert to uM
    N2sat = Henry[2]*(.781)
    
    if StartBool:
        StartBool = False

        refmodel.reactions.get_by_id("EX_cpd00011[e]").lower_bound = -RLPC(CO2sat*4,O2sat,T)
        Inputs['refsolution'] = refmodel.optimize()
        Inputs['LightLimit'] = Inputs['refsolution'].fluxes["EX_PHO1"]+Inputs['refsolution'].fluxes["EX_PHO2"]
        Inputs['NH4Constraint'] = -4.5*Inputs['refsolution'].fluxes["EX_cpd00528[e]"]
        Inputs['prevy'] = y
        Inputs['prevdydt'] = [0, 0, 0, 0, 0, 0]
    
    if (date.hour) == 0 and daycounter != elapsedday:
        daycounter += 1
        FirstDay = True

        refmodel.reactions.get_by_id("EX_cpd00011[e]").lower_bound = -RLPC(CO2sat*4,O2sat,T)
        Inputs['refsolution'] = refmodel.optimize()
        Inputs['LightLimit'] = Inputs['refsolution'].fluxes["EX_PHO1"]+Inputs['refsolution'].fluxes["EX_PHO2"]
        Inputs['NH4Constraint'] = -4.5*Inputs['refsolution'].fluxes["EX_cpd00528[e]"]

        BioConstraint = ML.problem.Constraint(
            (ML.reactions.DM_biomass_c.flux_expression - ML.reactions.biomass_eq_33047__vc.flux_expression),
            lb=0,
            ub=0)
        ML.add_cons_vars(BioConstraint)

        NonTrivial =  ML.problem.Constraint(
            (ML.reactions.DM_biomass_c.flux_expression + ML.reactions.biomass_eq_33047__vc.flux_expression),
            lb=5e-6,
            ub=1000)
        ML.add_cons_vars(NonTrivial)

        Imax = ML.problem.Constraint(
            (ML.reactions.EX_PHO1.flux_expression + ML.reactions.EX_PHO2.flux_expression),
            lb=-1000,
            ub=0)
        ML.add_cons_vars(Imax)

        absorbML = {'EX_photon410_e': 4730.075289,'EX_photon430_e': 5817.128965,'EX_photon450_e': 5348.203973,'EX_photon470_e': 4050.000013,
                    'EX_photon490_e': 3464.694801,'EX_photon510_e': 2649.794528,'EX_photon530_e': 1876.490736,'EX_photon550_e': 1334.544022,
                    'EX_photon570_e': 873.4095179,'EX_photon590_e': 740.7816246,'EX_photon610_e': 888.7175101,'EX_photon630_e': 1082.718272,
                    'EX_photon650_e': 1178.924274,'EX_photon670_e': 3322.974688,'EX_photon690_e': 1840.91646}
        for k in absorbML.keys():
            ML.reactions.get_by_id(k).upper_bound = 0
            ML.reactions.get_by_id(k).lower_bound = -1000
        
        # Allow open fluxes for limiting nutrients to use their fluxes as flags for when the LP needs to be calculated.
        ML.reactions.EX_co2_e.lower_bound = -RLPD(CO2sat*4,O2sat,T)
        ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound = -RLPC(CO2sat*4,O2sat,T)
        ML.reactions.EX_no3_e.lower_bound = -1000
        ML.reactions.EX_sio4h4_e.lower_bound = -1000
        flagsolution = ML.optimize()
      
        ML.remove_cons_vars(BioConstraint)
        ML.remove_cons_vars(NonTrivial)
        ML.remove_cons_vars(Imax)
        
        Inputs["NO3flag"] = scipy.optimize.fsolve(NO3func,1,args=(flagsolution.fluxes["EX_no3_e"]))[0]
        Inputs["SiOH4flag"] = scipy.optimize.fsolve(SiOH4func,1,args=(flagsolution.fluxes["EX_sio4h4_e"]))[0]   

    # Create an object that will store data for wavelength-dependent light transmission through the atmosphere.
    ICalc = TotSolEng(Inputs)
    # Calculate the total incident extraterrestrial radiation (ETR) based on date, time and location.
    ICalc.CalcTotSolarEng(Inputs)
    # Calculate wavelength-dependent light transmission based on ETR spectrum.
    ICalc.CalcSolarSpec(Inputs)
    
    # Calculate wavelength-dependent light absorption/scattering by the ocean at a certain depth.
    ICalc.waterabs(0)
    ICalc.DTOTphoto = np.asarray(ICalc.DTOTphoto).squeeze()

    deathK = 1/((13-(T*0.25))*24)
    if y[5] < 0.1:
        deathKcyano = 1/((15-(T*0.25))*24)
    else:
        deathKcyano = 0
    
    NightCheck = False
    Itot = scipy.integrate.simpson(ICalc.DTOTphoto,ICalc.allWVLphoto)
    
    # Integrate only the photoactive region of the incident spectrum to get total photoactive light intensity.
    Iphoto = par(ICalc.DTOTphoto,ICalc.allWVLphoto)
        
    if Itot < -2*Inputs["LightLimit"]:
        NightCheck = True
        
    if y[4] < Inputs["NO3flag"] and y[4] > 0.01:
        NO3flag = True
    else:
        NO3flag = False

    if y[5] < Inputs["SiOH4flag"] and y[5] > 0.1:
        SiOH4flag = True
    else:
        SiOH4flag = False
    
    print("Latitude:", str(Inputs['Latitude']))
    print("Date:", date)
    print("Total Light:", Itot)
    print("Diatom Biomass:",y[0])
    print("Cyanobac Biomass:",y[1])
    print("Chrysolaminarin Concentration:",y[2])
    print("Nitrate Concentration:",y[4])
    print("Silicon Concentration:",y[5])
    print("Day Flag:", FirstNight)
    print("Night Flag:", FirstDay)
    print("NO3 Flag:", NO3flag)
    print("SiOH4 Flag:", SiOH4flag)
    print("NightChecker:", -2*Inputs["LightLimit"])
    print("Time Elapsed:", t)
    clear_output(wait=True)

    if not FirstLight:
        if Itot < -2*Inputs["LightLimit"]:
            return [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        else:
            FirstLight = True

    if y[3] > CO2sat:
        y[3] = CO2sat
    
    KLaCO2 = 10
    diffCO2 = KLaCO2*(CO2sat-y[3])
    
    ### DIATOM LIMITING RESOURCES CALCULATIONS. 
    # Defaults for co2 exchange are ub = 0, lb = 0. Unbounded for now.
    ML.reactions.EX_co2_e.upper_bound = 1000
    ML.reactions.EX_co2_e.lower_bound = -RLPD(CO2sat*4,O2sat,T)
    hco3 = 53.70*CO2sat # buffer calculation for hco3, assuming pH = 8.1 with pKa = 6.37 [10^(8.1-6.37) = 53.70]
    chla = 0.192e-9
    DW = 16.62883198e-12
    n = 1.9
#    ML.reactions.EX_hco3_e.lower_bound = -((2.438*hco3)/(258.6+hco3)) # Trimborn, 2009
    ML.reactions.EX_hco3_e.lower_bound = 0
    if y[4] > 0.01:
        ML.reactions.EX_no3_e.lower_bound = -((1.474*y[4])/(6.14+y[4]))
    else:
        y[4] = 0
        ML.reactions.EX_no3_e.lower_bound = 0
    
    if y[5] > 0.1:
        ML.reactions.EX_sio4h4_e.lower_bound = -((1.961*(y[5]**n))/((8.1**n)+(y[5]**n)))
    else:    
        y[5] = 0
        ML.reactions.EX_sio4h4_e.lower_bound = 0

    # LP Calculation Flags, abort loop here if no flags are triggered and use previous LP results.

    if NightCheck and FirstNight:
        FirstNight = False
        FirstDay = True
    elif NightCheck and not FirstNight and not NO3flag and not SiOH4flag and not len(Inputs['prevdydt']) == 6:
        return [x*(float(y[2])/Inputs['prevy'][2]) for x in Inputs['prevdydt']]
    elif not NightCheck and FirstDay:
        FirstDay = False
        FirstNight = True
    elif not NightCheck and not FirstDay and not NO3flag and not SiOH4flag:
        if (Inputs["prevdydt"][0]-deathK)*y[0] > 0:
            Inputs["prevdydt"][2] = ((0.0085*math.exp(0.0163*Itot))*y[0])
        return Inputs['prevdydt']
    elif not NO3flag and Inputs['prevdydt'][4] < 0.975*Inputs['NO3flag']:
        pass

        # Abort and Use Previous LP Calculation   
    
### DIATOM LIGHT ABSORPTION CALCULATIONS.
    for k in Iphoto.keys():
        ML.reactions.get_by_id(k).upper_bound = 0
        ML.reactions.get_by_id(k).lower_bound = Iphoto[k]*-1.
        ML.reactions.get_by_id(k).upper_bound = Iphoto[k]*-0.9999
    
    ### CYANOBACTERIA CALCULATIONS
    
    # Remove author's biomass constraint forcing a growth rate at or above observed.
    # If this biomass constraint is used, all solutions at low light are infeasible.
    ML.reactions.biomass_eq_33047__hc.lower_bound = 0
    
    # Rubisco-limited photosynthesis 
    ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound = -RLPC(CO2sat*4,O2sat,T)

    # Turn off HCO3 intake.
    ML.reactions.get_by_id("EX_cpd00242[e]").lower_bound = 0
    
    # Nitrogen fixation, including boost by being symbiotic with diatom...
    ML.reactions.get_by_id("EX_cpd00528[e]").lower_bound = -(0.9*N2sat)/(165+N2sat)
    
    # Check for nighttime!
    if NightCheck:
        
        if y[2] > 0 and y[4] > 10^-2:

            ML.reactions.nh4T.lower_bound = 0
            ML.reactions.nh4T.upper_bound = 1000
            ML.reactions.carbonT.lower_bound = 0
            ML.reactions.carbonT.upper_bound = 1000
            
            # Shut down or turn on the chrysolaminarin reaction.
            ML.reactions.EX_chryso_e.lower_bound = -0.18*y[2]/y[0]
            # Allow nitrate uptake by cyanobac at night to handle chrysolaminarin respiration.
            if y[4] > 10^-2:
                ML.reactions.get_by_id("EX_cpd00209[e]").lower_bound = -((1.474*y[4])/(6.14+y[4]))
            elif y[4] < 0:
                y[4] = 0
                ML.reactions.get_by_id("EX_cpd00209[e]").lower_bound = 0
            else:
                ML.reactions.get_by_id("EX_cpd00209[e]").lower_bound = 0

            ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound = 0
            ML.reactions.get_by_id("EX_cpd00242[e]").lower_bound = 0
            ML.reactions.EX_co2_e.lower_bound = 0
            
            for k in Iphoto.keys():
                ML.reactions.get_by_id(k).upper_bound = 0
                ML.reactions.get_by_id(k).lower_bound = -1
                ML.reactions.get_by_id(k).upper_bound = -.99
            
            Imax = ML.problem.Constraint(
                (ML.reactions.EX_PHO1.flux_expression + ML.reactions.EX_PHO2.flux_expression),
                lb=-1,
                ub=0)
            ML.add_cons_vars(Imax)
            
            BioConstraint = ML.problem.Constraint(
                (ML.reactions.DM_biomass_c.flux_expression - ML.reactions.biomass_eq_33047__vc.flux_expression),
                lb=0,
                ub=0)
            ML.add_cons_vars(BioConstraint)

            NonTrivial =  ML.problem.Constraint(
                (ML.reactions.DM_biomass_c.flux_expression + ML.reactions.biomass_eq_33047__vc.flux_expression),
                lb=5e-6,
                ub=1000)
            ML.add_cons_vars(NonTrivial)
            
            solution = ML.optimize()
            
            ML.remove_cons_vars(BioConstraint)
            ML.remove_cons_vars(Imax)
            ML.remove_cons_vars(NonTrivial)
            
            dydt = [(solution.fluxes["DM_biomass_c"]-deathK)*y[0]] # Live Diatom Biomass
            dydt.append((solution.fluxes["biomass_eq_33047__vc"]-deathK-deathKcyano)*y[1]) # Live Cyanobacteria Biomass
            dydt.append(-solution.fluxes["chrysoT"]*y[0]) # Chyrsolaminarin
            dydt.append(solution.fluxes["EX_co2_e"]*y[0]+solution.fluxes["EX_cpd00242[e]"]*y[1]+diffCO2) # Carbon Dioxide
            if solution.fluxes["EX_cpd00209[e]"] > 0:
                dydt.append(solution.fluxes["EX_no3_e"]*y[0]+solution.fluxes["EX_cpd00209[e]"]*y[1])
            else:
                dydt.append(solution.fluxes["EX_no3_e"]*y[0])
            dydt.append(solution.fluxes["EX_sio4h4_e"]*y[0])            
            dydt.append(solution.fluxes["DM_biomass_c"]*y[0]+solution.fluxes["biomass_eq_33047__vc"]*y[1]) # Biomass without death constant
            dydt.append(-solution.fluxes["EX_co2_e"]*y[0]-solution.fluxes["EX_cpd00242[e]"]*y[1]) # Total CO2 fixed
            dydt.append(solution.fluxes["EX_o2_e"]*y[0]+solution.fluxes["EX_cpd00007[e]"]*y[1]) # Total O2 produced
            dydt.append(-solution.fluxes["EX_cpd00528[e]"]*y[1]) # Total N2 fixed

            Inputs['prevdydt'] = dydt
            Inputs['prevy'] = y
        
        else:
            
            dydt = [-deathK*y[0], -deathK*y[1], 0, 0, 0, 0, 0, 0, 0, 0]
            return dydt
            
    # Otherwise, it is daytime.
    else:
        
        Imax = ML.problem.Constraint(
            (ML.reactions.EX_PHO1.flux_expression + ML.reactions.EX_PHO2.flux_expression),
            lb=-Itot,
            ub=0)
        ML.add_cons_vars(Imax)
        
        AmmoniaConstraint = ML.problem.Constraint(
            ML.reactions.nh4T.flux_expression,
            lb=0,
            ub=Inputs['NH4Constraint'])
        ML.add_cons_vars(AmmoniaConstraint)
        
        ML.reactions.EX_chryso_e.lower_bound = 0
        ML.reactions.get_by_id("EX_cpd00209[e]").lower_bound = 0 

        if y[4] > 0.125:
            CT = np.interp(T,Tn,datamin[:,-1,-1])
            CarbonConstraint = ML.problem.Constraint(
                ML.reactions.carbonT.flux_expression,
                lb=-(CT/6)*ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound,
                ub=-(CT/6)*ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound)
            ML.add_cons_vars(CarbonConstraint)
            solution = ML.optimize()
            ML.remove_cons_vars(CarbonConstraint)
        elif y[5] > 0.75:
            CT = IYSi([T,y[4]])
            CarbonConstraint = ML.problem.Constraint(
                ML.reactions.carbonT.flux_expression,
                lb=-(CT/6)*ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound,
                ub=-(CT/6)*ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound)
            ML.add_cons_vars(CarbonConstraint)
            solution = ML.optimize()
            ML.remove_cons_vars(CarbonConstraint)
        elif y[5] > 0.32:
            CT = INSi([T,y[4],y[5]])
            CarbonConstraint = ML.problem.Constraint(
                ML.reactions.carbonT.flux_expression,
                lb=-(CT/6)*ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound,
                ub=-(CT/6)*ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound)
            ML.add_cons_vars(CarbonConstraint)
            solution = ML.optimize()
            ML.remove_cons_vars(CarbonConstraint)
        else:
            ML.reactions.carbonT.upper_bound = 0
            solution = ML.optimize()
        
        # Remove the current light constraint.
        ML.remove_cons_vars(Imax)
        ML.remove_cons_vars(AmmoniaConstraint)
        
        dydt = [(solution.fluxes["DM_biomass_c"]-deathK)*y[0]] # Live diatom biomass
        dydt.append((solution.fluxes["biomass_eq_33047__vc"]-deathK-deathKcyano)*y[1]) # Live cyanobacteria biomass
        if (solution.fluxes["DM_biomass_c"]-deathK)*y[0] > 0:
            dydt.append((0.0085*math.exp(0.0163*Itot))*y[0])   # Chrysolaminarin, change to biomass soon.
        else:
            dydt.append(0)
        dydt.append(solution.fluxes["EX_co2_e"]*y[0]+solution.fluxes["EX_cpd00242[e]"]*y[1]+diffCO2) # Carbon Dioxide
        dydt.append(solution.fluxes["EX_no3_e"]*y[0]) # Nitrate
        dydt.append(solution.fluxes["EX_sio4h4_e"]*y[0]) # Silicon
        dydt.append(solution.fluxes["DM_biomass_c"]*y[0]+solution.fluxes["biomass_eq_33047__vc"]*y[1]) # Biomass without death constant
        dydt.append(-solution.fluxes["EX_co2_e"]*y[0]-solution.fluxes["EX_cpd00011[e]"]*y[1]) # Total CO2 fixed
        dydt.append(solution.fluxes["EX_o2_e"]*y[0]+solution.fluxes["EX_cpd00007[e]"]*y[1]) # Total O2 produced
        dydt.append(-solution.fluxes["EX_cpd00528[e]"]*y[1]) # Total N2 fixed

        Inputs['prevdydt'] = dydt
    
    return dydt

In [ ]:
global daycounter
global StartBool
global FirstDay
global FirstNight
global FirstLight

#lats = [55, 57.5, 60, 62.5, 65, 67.5, 70, 72.5, 75]
lats = [55, 65, 75, 85]
#lats = [70]
time = []
latmap = []
Dbiomass = []
Cbiomass = []
NO3 = []
SiO4H4 = []
totalBM = []
totalCO2 = []
totalO2 = []
totalN2 = []

for idx,lat in enumerate(lats):

    ML, refmodel, IYSi, INSi = LoadModel()
    
    print("Latitude: " + str(lat))
    daycounter = 0
    StartBool = True
    FirstDay = True
    FirstNight = False
    FirstLight = False
    
    temp = np.loadtxt('sstdailyMarAug2020.csv',delimiter=',')[:,idx*4]
    Henry = henry(int(temp[0]))
    CO2sat = Henry[0]*(4.21e-4)
    Inputs['Latitude'] = lat

    refmodel.reactions.get_by_id("EX_cpd00011[e]").lower_bound = -RLPC(CO2sat*4,O2sat,5)
    refmodel.reactions.get_by_id("EX_cpd00242[e]").lower_bound = 0
    refmodel.reactions.get_by_id("EX_cpd00027[e]").lower_bound = 0
    refmodel.reactions.biomass_eq_33047__hc.lower_bound = 0
    
    IVPSol = scipy.integrate.solve_ivp(dydt,[0, 4300],[0.0005, 0.0005, 0.055, 27, 10, 10, 0.001, 0, 0, 0],max_step=1,args=[Inputs,start_date,temp,IYSi,INSi,Tn,datamin])
    #IVPSol = scipy.integrate.solve_ivp(dydt,[0, 720],[7.472547668746805, 8.554385159333298, 180.5802403287163, 27, 0, 0.28245253147206567, 0, 0, 0, 0],max_step=1,args=[Inputs,start_date,temp,IAll,Tn,datamin])
    time.append(IVPSol['t'])
    
    latinput = np.zeros((1,len(IVPSol['t'])))
    latinput.fill(lat)
    latmap.append(latinput)
    
    Dbiomass.append(IVPSol['y'][0])
    Cbiomass.append(IVPSol['y'][1])
    NO3.append(IVPSol['y'][4])
    SiO4H4.append(IVPSol['y'][5])
    totalBM.append(IVPSol['y'][6])
    totalCO2.append(IVPSol['y'][7])
    totalO2.append(IVPSol['y'][8])
    totalN2.append(IVPSol['y'][9])
    
timeflat = listarraytoarray(time)
latflat = flattenlat(latmap)
Dbioflat = listarraytoarray(Dbiomass)
Cbioflat = listarraytoarray(Cbiomass)
NO3flat = listarraytoarray(NO3)
SiO4H4flat = listarraytoarray(SiO4H4)
totalBMflat = listarraytoarray(totalBM)
totalCO2flat = listarraytoarray(totalCO2)
totalO2flat = listarraytoarray(totalO2)
totalN2flat = listarraytoarray(totalN2)

output = np.array([timeflat, latflat, Dbioflat, Cbioflat, NO3flat, SiO4H4flat, totalBMflat, totalCO2flat, totalO2flat, totalN2flat])
np.savetxt("surfplotAprMayDoubleNew.csv",output,delimiter=',')

In [ ]:
March = np.loadtxt("surfplotAprMayDoubleNew.csv",delimiter=",")
lengths = []
#lats = [65, 67.5, 70, 72.5, 75, 77.5, 80, 82.5, 85]
lats = [55, 65, 75, 85]
#lats = [75, 85]
#lats = [70]

numrows = len(lats)

for lat in lats:
    lengths.append(len(np.where(March[1] == lat)[0]))

arraydict = {}
arraydict['arraytime'] = np.zeros((numrows,max(lengths)))
arraydict['arraylat'] = np.zeros((numrows,max(lengths)))
arraydict['arrayDbio'] = np.zeros((numrows,max(lengths)))
arraydict['arrayCbio'] = np.zeros((numrows,max(lengths)))
arraydict['arrayNO3'] = np.zeros((numrows,max(lengths)))
arraydict['arraySiO4H4'] = np.zeros((numrows,max(lengths)))
arraydict['arraytotalBM'] =  np.zeros((numrows,max(lengths)))
arraydict['arraytotalCO2'] =  np.zeros((numrows,max(lengths)))
arraydict['arraytotalO2'] =  np.zeros((numrows,max(lengths)))
arraydict['arraytotalN2'] =  np.zeros((numrows,max(lengths)))
count = 0

for entry in arraydict.keys():
    
    for idx, lat in enumerate(lats):
        ids = np.where(March[1] == lat)
        hold = March[count][ids[0][0]:ids[0][-1]]
        holdpad = np.pad(hold,(0,max(lengths)-len(ids[0])+1),'constant',constant_values=(0,hold[-1]))
        arraydict[entry][idx,:] = holdpad
        
    count += 1

In [ ]:
x = arraydict['arraytime'] # Time
y = arraydict['arraylat'] # Latitude
variables = ['arrayDbio','arrayNO3','arraySiO4H4','arraytotalBM']
zlabels = ["Active Biomass \n (mgDW/L)", "Nitrate Concentration \n (umol/L)",
           "Silicic Acid Concentration \n (umol/L)","Total Biomass \n (mgDW/L)"]

ax = []

fig = plt.figure(figsize=(9.5,9.5))

ax.append(fig.add_subplot(221,projection='3d'))
ax.append(fig.add_subplot(222,projection='3d'))
ax.append(fig.add_subplot(223,projection='3d'))
ax.append(fig.add_subplot(224,projection='3d'))


for idx, obj in enumerate(ax):
    
    obj.set_xlabel("Time (hrs)",fontsize=8)
    obj.set_ylabel("Latitude (deg N)",fontsize=8)
    obj.set_zlabel(zlabels[idx],fontsize=8,labelpad=2)
    obj.set_yticks([65,70,75,80,85])
    obj.set_zticks([4,8,12,16,20])
    obj.tick_params(axis='both',which='both',labelsize=8)
    obj.plot_surface(x,y,arraydict[variables[idx]])
    
ax[0].view_init(20,260)
ax[1].view_init(20,280)
ax[2].view_init(20,280)
ax[3].view_init(20,260)

plt.savefig("MayDiatomdFBADouble.pdf",dpi=600,format='pdf')

In [ ]:
x = arraydict['arraytime'] # Time
# y = arraydict['arraylat'] # Latitude
#lats = [65, 67.5, 70, 72.5, 75, 77.5, 80, 82.5, 85]
lats = [75, 85]
variables = ['arrayDbio','arrayNO3','arraySiO4H4','arraytotalBM']
labels = ["Active Biomass \n (mgDW/L)", "Nitrate Concentration \n (umol/L)",
           "Silicic Acid Concentration \n (umol/L)","Total Biomass \n (mgDW/L)"]

ax = []

fig = plt.figure(figsize=(9.5,9.5))

ax.append(fig.add_subplot(221))
ax.append(fig.add_subplot(222))
ax.append(fig.add_subplot(223))
ax.append(fig.add_subplot(224))

for idx, obj in enumerate(ax):
    
    obj.set_xlabel("Time (hrs)",fontsize=8)
    obj.set_ylabel(labels[idx],fontsize=8)
    #obj.set_zlabel(zlabels[idx],fontsize=8,labelpad=2)
    fig.legend(["55oN","65oN","75oN","85oN"],loc="center right")
    
    for idy, lat in enumerate(lats):
        obj.scatter(x[idy,:],arraydict[variables[idx]][idy,:],s=0.5)

plt.savefig("MayDiatomdFBADouble.pdf",dpi=600,format='pdf')

In [ ]:
x = arraydict['arraytime']
lats = [55,65,75,85]
variables = ['arrayDbio','arrayCbio','arrayNO3','arraySiO4H4','arraytotalBM','arraytotalCO2','arraytotalO2','arraytotalN2']
labels = ["Active Diatom Biomass \n (mgDW/L)", "Active Cyanobacteria Biomass \n (mgDW/L)", "Nitrate Concentration \n (umol/L)",
           "Silicic Acid Concentration \n (umol/L)","Total Biomass Generated \n (mgDW/L)", "Total CO2 Consumed \n (umol/L)",
           "Total O2 Produced \n (umol/L)", "Total N2 Consumed \n (umol/L)"]
filenames = ["DiatomBio","CyanoBio","Nitrate","Silicic","TotalBio","TotalCO2","TotalO2","TotalN2"]


for idx, name in enumerate(filenames):
    plt.figure()
            
    for idy, lat in enumerate(lats):
        plt.scatter(x[idy,:],arraydict[variables[idx]][idy,:],s=0.75)

    plt.xlabel("Time (hours)")
    plt.ylabel(labels[idx]) 

    plt.rc('font', size=16)          # controls default text sizes
    plt.rc('axes', titlesize=18)     # fontsize of the axes title
    plt.rc('axes', labelsize=16)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=16)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=16)    # fontsize of the tick labels
    plt.rc('legend', fontsize=16)    # legend fontsize
    plt.rc('figure', titlesize=12)  # fontsize of the figure title
    plt.savefig(("March-AugustNewBigNoLegend " + name + ".pdf"),dpi=1200,format='pdf')

In [ ]:
idx = 1

plt.figure()

plt.scatter(arraydict['arraytime'][idx,:],arraydict['arrayDbio'][idx,:],s=0.5)
plt.scatter(arraydict['arraytime'][idx,:],arraydict['arrayCbio'][idx,:],s=0.5)
plt.scatter(arraydict['arraytime'][idx,:],arraydict['arraySiO4H4'][idx,:],s=0.5)
plt.scatter(arraydict['arraytime'][idx,:],arraydict['arrayNO3'][idx,:],s=0.5)
plt.legend(["Diatom Biomass", "Cyanobac Biomass", "Silicon Concentration", "Nitrate Concentration"])
plt.xlabel("Time (hours)")
plt.ylabel("Concentration (Bio: mgDW/L, Chem: umol/L)")

plt.show()

In [ ]:
def CarbonTransfer(x, T, NO3, Sili):

    Henry = henry(T)
    CO2sat = Henry[0]*(4.21e-4) # Partial pressure of CO2 in atm. # convert to uM
    O2sat = Henry[1]*(.2105) # convert to uM
    N2sat = Henry[2]*(.781)

    refmodel.reactions.get_by_id("EX_cpd00011[e]").lower_bound = -RLPC(CO2sat*4,O2sat,T)
    refmodel.reactions.get_by_id("EX_cpd00242[e]").lower_bound = 0
    refmodel.reactions.get_by_id("EX_cpd00027[e]").lower_bound = 0
    refmodel.reactions.biomass_eq_33047__hc.lower_bound = 0

    refsolution = refmodel.optimize()

    #ML.solver.configuration.verbosity = 3
    ML.solver.configuration.timeout = 5
    ML.solver.configuration.tolerance_feasibility = 1e-5

    ML.reactions.EX_co2_e.upper_bound = 1000
    ML.reactions.EX_co2_e.lower_bound = -RLPD(CO2sat,O2sat,T)
    hco3 = 53.70*CO2sat
    ML.reactions.EX_hco3_e.lower_bound = -((2.438*hco3)/(258.6+hco3))

    ML.reactions.EX_no3_e.lower_bound = -((1.474*NO3)/(6.14+NO3))

    ML.reactions.EX_sio4h4_e.lower_bound = -((1.961*(Sili**n))/((8.1**n)+(Sili**n)))
    
    ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound = -RLPC(CO2sat*4,O2sat,T)
    ML.reactions.get_by_id("EX_cpd00528[e]").lower_bound = -(0.9*N2sat)/(165+N2sat)

    Imax = ML.problem.Constraint(
        (ML.reactions.EX_PHO1.flux_expression + ML.reactions.EX_PHO2.flux_expression),
        lb=-Itot,
        ub=0)
    ML.add_cons_vars(Imax)
    
    AmmoniaConstraint = ML.problem.Constraint(
        ML.reactions.nh4T.flux_expression,
        lb=-4.5*refsolution.fluxes["EX_cpd00528[e]"],
        ub=-4.5*refsolution.fluxes["EX_cpd00528[e]"])
    ML.add_cons_vars(AmmoniaConstraint)
    
    CarbonConstraint = ML.problem.Constraint(
        ML.reactions.carbonT.flux_expression,
        lb=-(x/6)*ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound,
        ub=-(x/6)*ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound)
    ML.add_cons_vars(CarbonConstraint)
    
    solution = ML.optimize()
    
    ML.remove_cons_vars(Imax)
    ML.remove_cons_vars(AmmoniaConstraint)
    ML.remove_cons_vars(CarbonConstraint)

    return (solution.fluxes["DM_biomass_c"]-solution.fluxes["biomass_eq_33047__vc"])

In [ ]:
Tn = np.linspace(0,8,8)
NO3 = np.linspace(0,0.125,2)
Sili = np.linspace(0.3,1.5,5)

datamin = np.zeros((len(Tn),len(NO3),len(Sili)))

for idx, temp in enumerate(Tn):
                   
    for idy, NO3ele in enumerate(NO3):

        for idz, Siliele in enumerate(Sili):
            resbrute = []
            counter = 0

            linearrange = np.linspace(0,5,150)
            #linearrange = np.linspace(4-(idx*(0.2))-((5-idz)*(0.6)),5-(idx*(0.125))-((4-idz)*(0.6)),150+(idx))
                
            for idw, C in enumerate(linearrange):
                
                sol = CarbonTransfer(C,temp,NO3ele,Siliele)
                resbrute.append(abs(sol))
    
                print("Temperature:", idx+1)
                print("NO3:", idy+1)
                print("Silicon:", idz+1)
                print("Carbon Iteration:", idw+1)
                print(sol)
                clear_output(wait=True)
            
                counter += 1
    
            index = resbrute.index(min(resbrute))
            datamin[idx,idy,idz] = linearrange[index]

In [13]:
def CarbonTransferAbs(T, NO3,Sili):

    Henry = hencry(T)
    CO2sat = Henry[0]*(4.21e-4) # Partial pressure of CO2 in atm. # convert to uM
    O2sat = Henry[1]*(.2105) # convert to uM
    N2sat = Henry[2]*(.781)
    
    n=1.9

    refmodel.reactions.get_by_id("EX_cpd00011[e]").lower_bound = -RLPC(CO2sat*4,O2sat,T)
    refmodel.reactions.get_by_id("EX_cpd00242[e]").lower_bound = 0
    refmodel.reactions.get_by_id("EX_cpd00027[e]").lower_bound = 0
    refmodel.reactions.get_by_id("EX_cpd00528[e]").lower_bound = -(0.9*N2sat)/(165+N2sat)
    refmodel.reactions.biomass_eq_33047__hc.lower_bound = 0

    refsolution = refmodel.optimize()

#    ML.solver.configuration.verbosity = 3
    ML.solver.configuration.timeout = 5
    ML.solver.configuration.tolerance_feasibility = 1e-5

    ML.reactions.EX_co2_e.upper_bound = 1000
    ML.reactions.EX_co2_e.lower_bound = -RLPD(CO2sat*4,O2sat,T)
    hco3 = 53.70*CO2sat
    ML.reactions.EX_hco3_e.lower_bound = 0
#    ML.reactions.EX_hco3_e.lower_bound = -((2.438*hco3)/(258.6+hco3))

    ML.reactions.EX_no3_e.lower_bound = -((1.474*NO3)/(6.14+NO3))

    ML.reactions.EX_sio4h4_e.lower_bound = -((1.961*(Sili**n))/((8.1**n)+(Sili**n)))

    ML.reactions.get_by_id("EX_cpd00011[e]").upper_bound = 1000
    ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound = -RLPC(CO2sat*4,O2sat,T)
    ML.reactions.get_by_id("EX_cpd00011[e]").upper_bound = -RLPC(CO2sat*4,O2sat,T)*0.9999
    ML.reactions.get_by_id("EX_cpd00528[e]").lower_bound = -(0.9*N2sat)/(165+N2sat)

    Imax = ML.problem.Constraint(
        (ML.reactions.EX_PHO1.flux_expression + ML.reactions.EX_PHO2.flux_expression),
        lb=-Itot,
        ub=0)
    ML.add_cons_vars(Imax)
    
    AmmoniaConstraint = ML.problem.Constraint(
        ML.reactions.nh4T.flux_expression,
        lb=-4.5*refsolution.fluxes["EX_cpd00528[e]"],
        ub=-4.5*refsolution.fluxes["EX_cpd00528[e]"])
    ML.add_cons_vars(AmmoniaConstraint)
    
    BioConstraint = ML.problem.Constraint(
        (ML.reactions.DM_biomass_c.flux_expression - ML.reactions.biomass_eq_33047__vc.flux_expression),
        lb=0,
        ub=0)
    ML.add_cons_vars(BioConstraint)
    
 #   NonTrivial =  ML.problem.Constraint(
  #      (ML.reactions.DM_biomass_c.flux_expression + ML.reactions.biomass_eq_33047__vc.flux_expression),
   #     lb=0.005,
    #    ub=1000)
    #ML.add_cons_vars(NonTrivial)
    
    solution = ML.optimize()
    
    ML.remove_cons_vars(Imax)
    ML.remove_cons_vars(AmmoniaConstraint)
    ML.remove_cons_vars(BioConstraint)
   # ML.remove_cons_vars(NonTrivial)
    print(solution.fluxes["carbonT"])
    print(ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound)
    print(ML.reactions.EX_co2_e.lower_bound)
    print(solution.fluxes["DM_biomass_c"])
    print(solution.fluxes["biomass_eq_33047__vc"])
    print(solution.fluxes["EX_cpd00011[e]"])
    
    return (6*solution.fluxes["carbonT"]/(-ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound))

In [14]:
Tn = np.linspace(-3,26,30)
NO3 = np.linspace(0,.5,3)
Sili = np.linspace(0.2,2,18)

datamin = np.zeros((len(Tn),len(NO3),len(Sili)))

for idx, temp in enumerate(Tn):
                   
    for idy, NO3ele in enumerate(NO3):

        for idz, Siliele in enumerate(Sili):
                
            sol = CarbonTransferAbs(temp,NO3ele,Siliele)
    
            print("Temperature:", Tn[idx])
            print("NO3:", idy)
            print("Si(OH)4:", idz)
            print(sol)
            clear_output(wait=True)
    
            datamin[idx,idy,idz] = sol

0.4773440496107643
-0.4470482284359538
-6.000840955740129
0.07828716320116638
0.07828716320116638
-0.4470482284359538
Temperature: 26.0
NO3: 2
Si(OH)4: 17
6.406611447012825


In [15]:
datamin_re = datamin.reshape(datamin.shape[0], -1)

In [16]:
np.savetxt("CarbonTransfer.csv",datamin_re,delimiter=',')

In [17]:
datamin = datamin_re.reshape(datamin_re.shape[0], datamin_re.shape[1] // len(Sili), len(Sili))

In [ ]:
INSi = RegularGridInterpolator((Tn,NO3,Sili),datamin)
IYSi = RegularGridInterpolator((Tn,NO3),datamin)
ITemp = np.interp(4.5,Tn,datamin[:,-1])

In [ ]:
IAll([5,0.25])

In [ ]:
# Calculate the saturation concentration of carbon dioxide (in mM).
Henry = henry(5)
CO2sat = Henry[0]*(4.21e-4) # Partial pressure of CO2 in atm. # convert to uM
O2sat = Henry[1]*(.2105) # convert to uM
N2sat = Henry[2]*(.781)

y = [0, 0, CO2sat*4, 10, 10]
### DIATOM LIGHT ABSORPTION CALCULATIONS.
for k in Iphoto.keys():
    ML.reactions.get_by_id(k).upper_bound = 0
    ML.reactions.get_by_id(k).lower_bound = Iphoto[k]*-1.
    ML.reactions.get_by_id(k).upper_bound = Iphoto[k]*-0.9999

# Run this cell if you wish to perform autotrophic growth only.
ML.reactions.EX_glc__D_e.lower_bound = 0
ML.reactions.EX_glc__D_e.upper_bound = 0

# Shut down or turn on the chrysolaminarin reaction.
ML.reactions.EX_chryso_e.lower_bound = 0

ML.reactions.EX_no2_e.lower_bound = 0
ML.reactions.EX_nh4_e.lower_bound = 0
ML.reactions.EX_pi_e.lower_bound = -1
ML.reactions.EX_so4_e.lower_bound = -1
ML.reactions.EX_cncbl3_e.lower_bound = -1

### DIATOM LIMITING RESOURCES CALCULATIONS. 
# Defaults for co2 exchange are ub = 0, lb = 0. Unbounded for now.
ML.reactions.EX_co2_e.upper_bound = 1000
ML.reactions.EX_co2_e.lower_bound = -RLPD(CO2sat,O2sat,5)
hco3 = 53.70*CO2sat # buffer calculation for hco3, assuming pH = 8.1 with pKa = 6.37 [10^(8.1-6.37) = 53.70]
chla = 0.192e-9
DW = 16.62883198e-12
n = 1.9
ML.reactions.EX_hco3_e.lower_bound = -((2.438*hco3)/(258.6+hco3)) # Trimborn, 2009
ML.reactions.EX_no3_e.lower_bound = -((1.474*y[3])/(6.14+y[3]))

if np.isnan(-((1.961*(y[4]**n))/((8.1**n)+(y[4]**n)))):
    pass
else:    
    ML.reactions.EX_sio4h4_e.lower_bound = -((1.961*(y[4]**n))/((8.1**n)+(y[4]**n)))

### CYANOBACTERIA CALCULATIONS

# Remove author's biomass constraint forcing a growth rate at or above observed.
# If this biomass constraint is used, all solutions at low light are infeasible.
ML.reactions.biomass_eq_33047__hc.lower_bound = 0

# Rubisco-limited photosynthesis 
ML.reactions.get_by_id("EX_cpd00242[e]").lower_bound = 0

# Rubisco-limited photosynthesis 
ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound = -RLPC(y[2],O2sat,5)
#ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound = 0

# Nitrogen fixation, including boost by being symbiotic with diatom...
ML.reactions.get_by_id("EX_cpd00528[e]").lower_bound = -(0.9*N2sat)/(165+N2sat)

In [10]:
T = 5
NO3 = 9
Sili = 4
Henry = henry(T)
CO2sat = Henry[0]*(4.21e-4) # Partial pressure of CO2 in atm. # convert to uM
O2sat = Henry[1]*(.2105) # convert to uM
N2sat = Henry[2]*(.781)

n = 1.9

refmodel.reactions.get_by_id("EX_cpd00011[e]").lower_bound = -RLPC(CO2sat*4,O2sat,T)
refmodel.reactions.get_by_id("EX_cpd00242[e]").lower_bound = 0
refmodel.reactions.get_by_id("EX_cpd00027[e]").lower_bound = 0
refmodel.reactions.get_by_id("EX_cpd00528[e]").lower_bound = -(0.9*N2sat)/(165+N2sat)
refmodel.reactions.biomass_eq_33047__hc.lower_bound = 0

refsolution = refmodel.optimize()

#    ML.solver.configuration.verbosity = 3
ML.solver.configuration.timeout = 5
ML.solver.configuration.tolerance_feasibility = 1e-5

ML.reactions.EX_co2_e.upper_bound = 1000
ML.reactions.EX_co2_e.lower_bound = -RLPD(CO2sat*4,O2sat,T)
hco3 = 53.70*CO2sat
ML.reactions.EX_hco3_e.lower_bound = 0
#    ML.reactions.EX_hco3_e.lower_bound = -((2.438*hco3)/(258.6+hco3))

ML.reactions.EX_no3_e.lower_bound = -((1.474*NO3)/(6.14+NO3))

ML.reactions.EX_sio4h4_e.lower_bound = -((1.961*(Sili**n))/((8.1**n)+(Sili**n)))

ML.reactions.get_by_id("EX_cpd00011[e]").upper_bound = 1000
ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound = -RLPC(CO2sat*4,O2sat,T)
ML.reactions.get_by_id("EX_cpd00011[e]").upper_bound = -RLPC(CO2sat*4,O2sat,T)*0.9999
ML.reactions.get_by_id("EX_cpd00528[e]").lower_bound = -(0.9*N2sat)/(165+N2sat)
ML.reactions.EX_nh4_e.lower_bound = 0
ML.reactions.EX_nh4_e.upper_bound = 0

Imax = ML.problem.Constraint(
    (ML.reactions.EX_PHO1.flux_expression + ML.reactions.EX_PHO2.flux_expression),
    lb=-Itot,
    ub=0)
ML.add_cons_vars(Imax)

AmmoniaConstraint = ML.problem.Constraint(
    ML.reactions.nh4T.flux_expression,
    lb=-4.5*refsolution.fluxes["EX_cpd00528[e]"],
    ub=-10*refsolution.fluxes["EX_cpd00528[e]"])
ML.add_cons_vars(AmmoniaConstraint)

BioConstraint = ML.problem.Constraint(
    (ML.reactions.DM_biomass_c.flux_expression - ML.reactions.biomass_eq_33047__vc.flux_expression),
    lb=0,
    ub=0)
ML.add_cons_vars(BioConstraint)

solution = ML.optimize()

ML.remove_cons_vars(Imax)
ML.remove_cons_vars(AmmoniaConstraint)
ML.remove_cons_vars(BioConstraint)
# ML.remove_cons_vars(NonTrivial)
print("Transferred Glucose: ",solution.fluxes["carbonT"])
print("Cyanobacteria CO2 Intake: ",ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound)
print("Diatom CO2 Intake: ",ML.reactions.EX_co2_e.lower_bound)
print("Diatom Biomass Flux: ",solution.fluxes["DM_biomass_c"])
print("Cyanobacteria Biomass Flux: ",solution.fluxes["biomass_eq_33047__vc"])
print("Saved Carbon Transfer Value: ",(6*solution.fluxes["carbonT"]/(-ML.reactions.get_by_id("EX_cpd00011[e]").lower_bound)))
print("Cyanobacteria Metabolism %: ",100*((solution.fluxes["carbonT"]/((-solution.fluxes["EX_cpd00011[e]"]/6)+solution.fluxes["carbonT"]))))
print("Cyanobacteria Nitrogen %: ",100*((solution.fluxes["nh4T"]-refsolution.fluxes["EX_cpd00528[e]"])/(-refsolution.fluxes["EX_cpd00528[e]"])))

Transferred Glucose:  0.05733972969373664
Cyanobacteria CO2 Intake:  -0.2888378085860543
Diatom CO2 Intake:  -0.9624207826304703
Diatom Biomass Flux:  0.01496357521152003
Cyanobacteria Biomass Flux:  0.01496357521152003
Saved Carbon Transfer Value:  1.191112686550935
Cyanobacteria Metabolism %:  54.361087581756664
Cyanobacteria Nitrogen %:  550.0000000000022


In [ ]:
solution = ML.optimize()
ML.summary()